In [ ]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression # for building a linear regression model
from pyearth import Earth
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import warnings
import pygam as gam
from pygam import LinearGAM,s,f
from statsmodels.gam.api import GLMGam, BSplines
from IPython.display import display
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv('result.csv')
df = df.set_index(pd.DatetimeIndex(df['Date']))
df.index.names = ["Date"]
df=df.drop(columns=['Date'])
display(df)

In [ ]:
X = df[['volatility_index','peak','through','contraction','expansion','housing_prices_index','growth_GDP','unemp','lag_vol','through','crisis']]
y1 = df['nr_of_cases']

#MARS linear dependent var

MARS_model = Earth(max_terms=500, max_degree=8)

MARS_model_fitted = MARS_model.fit(X, y1)

y_pred_mars = MARS_model_fitted.predict(X)
print(MARS_model_fitted.summary())

In [ ]:
from sklearn.metrics import mean_squared_error
error=mean_squared_error(y1,y_pred_mars)
error
plt.plot(X,y1,'r.')
#plt.plot(xTest,yTest,'b.')
plt.plot(X,y_pred_mars,'b.')
plt.xlabel('x')
plt.ylabel('nr of fraud cases')
plt.title('MARS regression')
plt.show()

In [ ]:
#before and after the hinge
ddft=df[['growth_GDP']].copy()
ddft.columns=['before_hinge']
h=1.65897
ix=ddft['before_hinge']<h
ddft['after_hinge']=ddft['before_hinge']
ddft.loc[ix,'after_hinge']=np.nan
ddft.loc[~ix, 'before_hinge']=np.nan

ddft3=df[['crisis']].copy()
ddft3.columns=['before_hinge']
ddft3['after_hinge']=ddft3['before_hinge']

ddft2=df[['volatility_index']].copy()
ddft2.columns=['before_hinge']
h=22.64
ix=ddft2['before_hinge']<h
ddft2['after_hinge']=ddft2['before_hinge']
ddft2.loc[ix,'after_hinge']=np.nan
ddft2.loc[~ix, 'before_hinge']=np.nan
ddft.plot()

In [ ]:
#1 & crisis & No & -299.68 (from the output, non-pruned hinge)
ddft3=df[['crisis']].copy()
ddft3.columns=['before_hinge']
ddft3['after_hinge']=ddft3['before_hinge']

plt.figure(figsize=(20,8))
ax=plt.subplot(111)
ddft=df.query('crisis==1').copy()

display((ddft['crisis']*-299).to_frame('H_1').plot(ax=ax, c='red', style='o'))

In [ ]:
#GAM 
X = dft[['volatility_index','peak','expansion','housing_prices_index','GDP','unemp','lag_vol','through','crisis']].values
y1 = dft['nr_of_cases'].values
gam = LinearGAM(s(0) + s(1) + s(2) + s(3)+s(4)+s(5)+s(6)+s(7)+s(8),callbacks=['diffs']).gridsearch(X, y1)
G = gam.summary()

#or 

#linear dependent variable
gam = LinearGAM(n_splines = 25, spline_order=4, callbacks=['diffs']).gridsearch(X,y1,lam=lams)
G = gam.summary()

In [ ]:
## partial dependence plotting 
plt.figure();
fig, axs = plt.subplots(1,9);

titles = ['volatility_index','peak','expansion','housing_prices_index','GDP','unemp','lag_vol','through','crisis']
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    if i == 0:
        ax.set_ylim(-110,110)
    ax.set_title(titles[i]);